### Gerekli Kütüphanelerin yüklenmesi

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, model_from_json
from tensorflow.keras.layers import Dense, Conv2D, Activation, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ModelCheckpoint

2025-02-01 16:40:51.799399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738417251.816024   28096 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738417251.820895   28096 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-01 16:40:51.837895: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Veri setinin okunması

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yusufkorayhasdemir/fer2013csv")

print("Path to dataset files:", path)

/home/dilaver/tf_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/dilaver/.cache/kagglehub/datasets/yusufkorayhasdemir/fer2013csv/versions/1


In [3]:
os.chdir(path)
!ls -l

total 294020
-rw-r--r-- 1 dilaver dilaver 301072766 Feb  1 15:42 fer2013.csv


In [4]:
data = pd.read_csv(path+"/fer2013.csv")

In [5]:
data.shape

(35887, 3)

In [6]:
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [7]:
data["Usage"].value_counts()

Usage
Training       28709
PublicTest      3589
PrivateTest     3589
Name: count, dtype: int64

## EĞİTİM VERİSİ İÇİN ÖN İŞLEME ADIMLARI

In [8]:
np.unique(data["Usage"].values.ravel())
print("Eğitim setindeki örnek sayısı : %d"%(len(data[data.Usage =="Training"])))
print("Eğitim setindeki PrivateTest sayısı : %d"%(len(data[data.Usage =="PrivateTest"])))
print("Eğitim setindeki PublicTest sayısı : %d"%(len(data[data.Usage =="PublicTest"])))

Eğitim setindeki örnek sayısı : 28709
Eğitim setindeki PrivateTest sayısı : 3589
Eğitim setindeki PublicTest sayısı : 3589


### Eğitim işlemi için veri kümesinde ayrılmış olan Training kısmını alıyoruz

In [9]:
train_data = data[data.Usage == "Training"] # Sadece eğitim örneklerini train_data içine aldık.

**Eğitim örneklerinin piksel değerleri bize tablo halinde yan yana verildiği için boşluklardan parse ederek liste olarak değişkene alalım.**

In [63]:
# 'pixels' sütunundaki verileri listeye dönüştür
train_pixels = train_data.pixels.str.split(" ").tolist()

In [74]:
# İçi içe listelerdeki string verileri int'e dönüştürme fonksiyonu
def convert_str_to_int(nested_list):
    for i in range(len(nested_list)):
        if isinstance(nested_list[i], list):  # Eğer öğe bir liste ise
            nested_list[i] = convert_str_to_int(nested_list[i])  # Rekürsif olarak iç listeyi dönüştür
        elif isinstance(nested_list[i], str):  # Eğer öğe bir string ise
            try:
                nested_list[i] = int(nested_list[i])  # String'i int'e dönüştür
            except ValueError:
                pass  # Eğer dönüşüm başarısızsa, değişiklik yapma
    return nested_list

In [75]:
# Örnek iç içe liste
nested_list = [['1', '2', '3'], ['4', '5'], ['6', '7', '8']]
print("Orijinal Liste:", nested_list)

# String değerleri integer'a dönüştürme
converted_list = convert_str_to_int(nested_list)
print("Dönüştürülmüş Liste:", converted_list)

Orijinal Liste: [['1', '2', '3'], ['4', '5'], ['6', '7', '8']]
Dönüştürülmüş Liste: [[1, 2, 3], [4, 5], [6, 7, 8]]


In [65]:
type(train_pixels)

list

In [66]:
train_pixels = convert_str_to_int(train_pixels)

In [73]:
#train_pixels= train_pixels.map(lambda x: int(x) if isinstance(x, str) else x)

In [67]:
train_pixels = pd.DataFrame(train_pixels, dtype=int)

In [68]:
train_images = train_pixels.values

In [70]:
train_images = train_images.astype(np.float16)

In [71]:
print(train_images)

[[ 70.  80.  82. ... 106. 109.  82.]
 [151. 150. 147. ... 193. 183. 184.]
 [231. 212. 156. ...  88. 110. 152.]
 ...
 [ 74.  81.  87. ... 188. 187. 187.]
 [222. 227. 203. ... 136. 136. 134.]
 [195. 199. 205. ...   6.  15.  38.]]


In [72]:
print(train_images.shape)

(28709, 2304)
